In [7]:
from azure.ai.ml import command, Input, MLClient, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute, Environment, Data, AzureBlobDatastore
from dotenv import load_dotenv
from azure.ai.ml.constants import AssetTypes, InputOutputModes

import os

In [8]:
load_dotenv()

SUBSCRIPTION_ID = os.environ.get("SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP")
WORKSPACE_NAME = os.environ.get("WORKSPACE_NAME")

In [9]:
# connect to AML
ml_client = MLClient(
    DefaultAzureCredential(),
    SUBSCRIPTION_ID,
    RESOURCE_GROUP,
    WORKSPACE_NAME
)

In [10]:
# get prepared dataset (should be saved as data asset inside the workspace )
data_asset = ml_client.data.get(name="rnn-model-preprocessed-sb", version="1")
data_asset.path

'azureml://subscriptions/ce96fbca-fc23-466f-87e4-9b8cb5316116/resourcegroups/rg-review-analysis-teamc/workspaces/aml-review-analysis-teamc1/datastores/amazondatastore/paths/UI/2024-04-03_131651_UTC/preped_data.csv/'

In [14]:
# example to train a job in AML - it works
job = command(
    inputs=dict(
        training_data=data_asset.path,
        registered_model_name="RNN-model-sb-1"
    ),
    code="./train",
    command="python train.py --prep_data ${{inputs.training_data}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="keras-env@latest",
    experiment_name="reviews_analysis__train",
    display_name="amazon_reviews__train",
    compute="cpu-cluster"
)

returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor the job here : ", aml_url)

Monitor the job here :  https://ml.azure.com/runs/plum_rainbow_nhkt9cg131?wsid=/subscriptions/ce96fbca-fc23-466f-87e4-9b8cb5316116/resourcegroups/rg-review-analysis-teamc/workspaces/aml-review-analysis-teamc1&tid=65810625-201f-44a9-ba8a-3af31d76a870
